# **Pre-Processing**

In [ ]:
!git clone https://github.com/google-research/albert

Cloning into 'albert'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 359 (delta 1), reused 1 (delta 0), pack-reused 353
Receiving objects: 100% (359/359), 239.23 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (233/233), done.



*   Here we need to git clone all necessary libraries to evaluate results on the BioASQ task. 

*   We also need to replace the old method (ix) with (iloc) .



In [ ]:
!git clone https://github.com/dmis-lab/biobert
!git clone https://github.com/BioASQ/Evaluation-Measures
!sed -i 's/ix/iloc/g' biobert/biocodes/transform_nbset2bioasqform.py

Cloning into 'biobert'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 332 (delta 30), reused 9 (delta 3), pack-reused 278
Receiving objects: 100% (332/332), 509.66 KiB | 6.21 MiB/s, done.
Resolving deltas: 100% (192/192), done.
Cloning into 'Evaluation-Measures'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 301 (delta 3), reused 13 (delta 3), pack-reused 268
Receiving objects: 100% (301/301), 1.92 MiB | 11.48 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
!pip install -r albert/requirements.txt

     |████████████████████████████████| 89 kB 3.9 MB/s 
     |████████████████████████████████| 1.2 MB 8.3 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=56350019c94024aa07634e20cfb2f165da6b09423d6040c3e9c82b0e6ef7ab77
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.12.0
    Uninstalling tensorflow-hub-0.12.0:
      Successfully uninstalled tensorflow-hub-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


- Establish the connection with TPU. All credits to ALBERT team for creating this code which can be found here https://github.com/google-research/albert/blob/master/albert_glue_fine_tuning_tutorial.ipynb

In [ ]:
%tensorflow_version 1.x
import os
import pprint
import json
import tensorflow as tf

assert "COLAB_TPU_ADDR" in os.environ, "ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!"
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
os.environ['TPU_ADDRESS'] ="grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = "2x2"
print("TPU address is", TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)


TensorFlow 1.x selected.
TPU address is grpc://10.5.67.218:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 7520807799155237263),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 244759522420812704),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3307970561091219626),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 225199789335918404),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 16383022445883760268),
 _DeviceAttributes(/job:tpu_wor

# **SQuAD2.0** (Skip this step if you plan to use our SQuAD checkpoint)

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2021-08-07 00:36:10--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   203MB/s    in 0.2s    

2021-08-07 00:36:11 (203 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2021-08-07 00:36:11--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [ ]:
!gcloud auth login

In [ ]:
!gsutil cp  train-v2.0.json gs://biom-transformers/SQuAD/train.json
!gsutil cp  dev-v2.0.json gs://biom-transformers/SQuAD/dev.json

Copying file://train-v2.0.json [Content-Type=application/json]...
-
Operation completed over 1 objects/40.2 MiB.                                     
Copying file://dev-v2.0.json [Content-Type=application/json]...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


Here we copy BioM-ELECTRA-xxlarge checkpoint from our google drive. You can get this checkpoint from this link https://drive.google.com/file/d/1-ARTcFGuEj8X9FEUK8adCslsSHTCoPVR/view?usp=sharing

In [ ]:
!cp drive/MyDrive/BioM-ALBERT-xxlarge.tar.gz .
!tar -xvf BioM-ALBERT-xxlarge.tar.gz

BioM-ALBERT-xxlarge/
BioM-ALBERT-xxlarge/PubMD-30k-clean.vocab
BioM-ALBERT-xxlarge/PubMD-30k-clean.model
BioM-ALBERT-xxlarge/model.ckpt-264500.index
BioM-ALBERT-xxlarge/model.ckpt-264500.data-00000-of-00001
BioM-ALBERT-xxlarge/model.ckpt-264500.meta
BioM-ALBERT-xxlarge/.ipynb_checkpoints/
BioM-ALBERT-xxlarge/config.json


- You need to copy the checkpoint folder to the google bucket if you are planning to use the TPU.

- To create a google bucket go to this link https://console.cloud.google.com/storage/browser

In [ ]:
!gsutil -m cp BioM-ALBERT-xxlarge/* gs://biom-transformers/BioM-ALBERT-PubMed/

Copying file://BioM-ALBERT-xxlarge/config.json [Content-Type=application/json]...
Copying file://BioM-ALBERT-xxlarge/model.ckpt-264500.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://BioM-ALBERT-xxlarge/model.ckpt-264500.index [Content-Type=application/octet-stream]...
Copying file://BioM-ALBE

- Training script to fine-tune BioM-ALBERT-xxlarge on SQuAD task. Please note that all feature files such as "train_feature_file", "predict_feature_file" and "predict_feature_file" are created by the training script so you just need to give the path to where the training script will store them.

In [ ]:
!python -m albert.run_squad_v2 \
  --albert_config_file=gs://biom-transformers/BioM-ALBERT-PubMed/config.json \
  --output_dir=gs://biom-transformers/BioM-ALBERT-PubMed/BioM-ALBERT-SQuAD/ \
  --train_file=gs://biom-transformers/SQuAD/train.json \
  --predict_file=gs://biom-transformers/SQuAD/dev.json \
  --train_feature_file=gs://biom-transformers/BioM-ALBERT-PubMed/BioM-ALBERT-SQuAD/train.tf \
  --predict_feature_file=gs://biom-transformers/BioM-ALBERT-PubMed/BioM-ALBERT-SQuAD/predict.tf \
  --predict_feature_left_file=gs://biom-transformers/BioM-ALBERT-PubMed/BioM-ALBERT-SQuAD/predict_left.tf \
  --init_checkpoint=gs://biom-transformers/BioM-ALBERT-PubMed/model.ckpt-264500 \
  --spm_model_file=gs://biom-transformers/BioM-ALBERT-PubMed/PubMD-30k-clean.model \
  --do_lower_case \
  --max_seq_length=384 \
  --doc_stride=128 \
  --max_query_length=64 \
  --do_train \
  --do_predict \
  --train_batch_size=40 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=2.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30 \
  --use_tpu \
  --tpu_name $TPU_ADDRESS

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0808 00:07:45.314606 140424446166912 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0808 00:07:45.314822 140424446166912 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0808 00:07:45.415912 140424446166912 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0808 00:07:45.416125 140424446166912 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0808 00:07:45.516952 140424446166912 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0808 00:07:45.517173 1404244461

- Our evaluation result shows that we achieved an 86.69 F1 score. This is different than what we reported (87.00) since we use a larger batch size (128). 

- Since we use a smaller batch size, we adjusted the learning rate to 5e-5.

# **BioASQ7B**

- Upload the pre-processed dataset of of BioASQ from this page https://github.com/dmis-lab/bioasq-biobert to this notebook.


- This is the direct link https://drive.google.com/open?id=1-KzAQzaE-Zd4jOlZG_7k7D4odqPI3dL1

In [ ]:
!unzip /content/BioASQ-67b-15Oct2019.zip

Archive:  /content/BioASQ-67b-15Oct2019.zip
   creating: BioASQ-6b/
   creating: BioASQ-6b/test/
   creating: BioASQ-6b/test/Appended-Snippet/
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-1-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-2-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-3-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-4-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-5-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-1-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-2-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-3-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-4-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioAS

- Download golden dataset from here 

http://participants-area.bioasq.org/accounts/login/?next=/Tasks/7b/goldenDataset/ (You need to register)

In [ ]:
!unzip /content/Task7BGoldenEnriched.zip

Archive:  /content/Task7BGoldenEnriched.zip
   creating: Task7BGoldenEnriched/
  inflating: Task7BGoldenEnriched/7B1_golden.json  
  inflating: Task7BGoldenEnriched/7B2_golden.json  
  inflating: Task7BGoldenEnriched/7B3_golden.json  
  inflating: Task7BGoldenEnriched/7B4_golden.json  
  inflating: Task7BGoldenEnriched/7B5_golden.json  


Here we are copying the dataset to our bucket.

In [ ]:
!gsutil cp BioASQ-7b/train/Full-Abstract/BioASQ-train-factoid-7b-full-annotated.json gs://biom-transformers/SQuAD/train.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-1.json gs://biom-transformers/SQuAD/test1.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-2.json gs://biom-transformers/SQuAD/test2.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-3.json gs://biom-transformers/SQuAD/test3.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-4.json gs://biom-transformers/SQuAD/test4.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-5.json gs://biom-transformers/SQuAD/test5.json

Copying file://BioASQ-7b/train/Full-Abstract/BioASQ-train-factoid-7b-full-annotated.json [Content-Type=application/json]...
/ [1 files][ 10.4 MiB/ 10.4 MiB]                                                
Operation completed over 1 objects/10.4 MiB.                                     
Copying file:///content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-1.json [Content-Type=application/json]...
/ [1 files][159.8 KiB/159.8 KiB]                                                
Operation completed over 1 objects/159.8 KiB.                                    
Copying file:///content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-2.json [Content-Type=application/json]...
/ [1 files][104.8 KiB/104.8 KiB]                                                
Operation completed over 1 objects/104.8 KiB.                                    
Copying file:///content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-3.json [Content-Type=application/json]...
/ [1 files][151.1 KiB/151.1 KiB] 

- We create those temp files to store our predictions temporarily before calculating the results of five batches

In [ ]:
!mkdir temp
!mkdir temp/preds
!mkdir temp/out

- Here we are only training our fine-tuned checkpoint ( BioM-ALBERT-xxlarge-SQuAD ) on training dataset of BioASQ7B . Then later we will use this fine-tuned checkpoint ( BioM-ALBERT-xxlarge-SQuAD-BioASQ7B ) to make prediction of all five batches of BioASQ7B using test dataset . 

- Here we follow the same hyperparameters choice that we reported in our paper except that we can't use 128 as a batch size since Google Colab has TPUv2-8 instead of TPUv3-8.

In [ ]:
!gsutil rm -r gs://biom-transformers/BioASQ7B/
!python3 -m albert.run_squad_v2 --albert_config_file=gs://biom-transformers/BioM-ALBERT-PubMed/config.json \
--output_dir=gs://biom-transformers/BioASQ7B/ \
--train_file=BioASQ-7b/train/Full-Abstract/BioASQ-train-factoid-7b-full-annotated.json \
--train_feature_file=gs://biom-transformers/BioASQ7B/ALBERT-BioASQ7B.tf_record \
--init_checkpoint=gs://biom-transformers/BioM-ALBERT-PubMed/BioM-ALBERT-SQuAD/model.ckpt-best \
--spm_model_file=gs://biom-transformers/BioM-ALBERT-PubMed/PubMD-30k-clean.model \
--do_lower_case \
--max_seq_length=384 \
--doc_stride=128 \
--max_query_length=64 \
--do_train \
--train_batch_size=40 \
--predict_batch_size=8 \
--learning_rate=1e-5 \
--num_train_epochs=3.0 \
--warmup_proportion=.1 \
--save_checkpoints_steps=5000 \
--n_best_size=20 \
--max_answer_length=30 \
--use_tpu=True \
--tpu_name=$TPU_ADDRESS

CommandException: No URLs matched: gs://biom-transformers/BioASQ7B/
INFO:tensorflow:loading sentence piece model
I0808 00:48:23.846184 139770572162944 tokenization.py:188] loading sentence piece model
W0808 00:48:25.304444 139770572162944 estimator.py:1994] Estimator's model_fn (<function v2_model_fn_builder.<locals>.model_fn at 0x7f1ea4c15c20>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'gs://biom-transformers/BioASQ7B/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.23.46.98:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experime

- The training script of ALBERT has a bug that will take the first checkpoint stored in the folder. In this case, it will be model.ckpt-0 checkpoint. However, when we fine-tune our models on BioASQ7B training dataset from the last step, we generate the checkpoint (model.ckpt-415) . So we have two checkpoints stored in the folder, both model.ckpt-0 and model.ckpt-415 . We need to remove model.ckpt-0 so the training script will make predictions from model.ckpt-415 . 

- Please note even if you choose the "--init_checkpoint" to be model.ckpt-415 and there is a "model.ckpt-0" in the same folder, the training script will initiate the prediction from model.ckpt-0 checkpoint. 

In [ ]:
!gsutil rm gs://biom-transformers/BioASQ7B/model.ckpt-0.*

Removing gs://biom-transformers/BioASQ7B/model.ckpt-0.data-00000-of-00001...
Removing gs://biom-transformers/BioASQ7B/model.ckpt-0.index...
Removing gs://biom-transformers/BioASQ7B/model.ckpt-0.meta...
/ [3 objects]                                                                   
Operation completed over 3 objects.                                              


- Here we loop through all five batches and store the result on result.txt folder using >> result.txt command at the end of this scirpt.
- You can ignore "KeyError: 'answers'" error .

In [ ]:
%%shell
rm  temp/* >/dev/null 2>&1
rm  temp/*/* >/dev/null 2>&1
rm result.txt >/dev/null 2>&1
for k in 1 2 3 4 5;
do rm temp/out/* >/dev/null 2>&1;
gsutil rm gs://biom-transformers/BioASQ7B/nbest_predictions.json >/dev/null 2>&1
gsutil rm gs://biom-transformers/BioASQ7B/null_odds.json >/dev/null 2>&1
gsutil rm gs://biom-transformers/BioASQ7B/predictions.json >/dev/null 2>&1
python3 -m albert.run_squad_v2   --albert_config_file=gs://biom-transformers/BioM-ALBERT-PubMed/config.json \
--output_dir=gs://biom-transformers/BioASQ7B/  \
--init_checkpoint=gs://biom-transformers/BioASQ7B/model.ckpt-415 \
--spm_model_file=gs://biom-transformers/BioM-ALBERT-PubMed/PubMD-30k-clean.model \
--max_seq_length=384 \
--doc_stride=128 \
--max_query_length=64   \
--do_train=False   \
--predict_batch_size=8   \
--n_best_size=20   \
--max_answer_length=30 \
--use_tpu=True \
--tpu_name=$TPU_ADDRESS \
--do_predict \
--train_file=BioASQ-7b/train/Full-Abstract/BioASQ-train-factoid-7b-full-annotated.json \
--train_feature_file=gs://biom-transformers/BioASQ7B/ALBERT-BioASQ7B.tf_record \
--predict_file=BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-"$k".json \
--predict_feature_file=gs://biom-transformers/BioASQ7B/BioASQ7B-predict-"$k".tfrecord \
--predict_feature_left_file=gs://biom-transformers/BioASQ7B/BioASQ7B-predict-left-"$k".tfrecord \
--do_lower_case
gsutil cp gs://biom-transformers/BioASQ7B/nbest_predictions.json temp/preds/nbest_prediction-"$k".json;
python3 biobert/biocodes/transform_nbset2bioasqform.py --nbest_path=temp/preds/nbest_prediction-"$k".json --output_path=temp/out;java -Xmx10G -cp $CLASSPATH:Evaluation-Measures/flat/BioASQEvaluation/dist/BioASQEvaluation.jar evaluation.EvaluatorTask1b -phaseB -e 5 Task7BGoldenEnriched/7B"$k"_golden.json temp/out/BioASQform_BioASQ-answer.json | awk '{printf "%.2f    %.2f     %.2f\n", $2*100, $3*100, $4*100}' >>result.txt;
done;

INFO:tensorflow:loading sentence piece model
I0808 13:19:47.310646 140233314973568 tokenization.py:188] loading sentence piece model
W0808 13:19:48.863120 140233314973568 estimator.py:1994] Estimator's model_fn (<function v2_model_fn_builder.<locals>.model_fn at 0x7f8a62e66050>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'gs://biom-transformers/BioASQ7B/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.5.67.218:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None,

In [ ]:
!echo "Result of BioASQ on Five Different Batches"
!echo "Strict   Lenient   MRR"
!cat  result.txt

Result of BioASQ on Five Different Batches
Strict   Lenient   MRR
51.28    53.85     52.14
64.00    72.00     67.33
48.28    62.07     53.74
58.82    82.35     68.82
37.14    54.29     43.90


Results of BioM-ELECTRA-Large which we obtained from other notebook example. We listed those results here to compare the perfomance between BioM-ELECTRA-Large and BioM-ALBERT-xxlarge.

Strict   Lenient   MRR

51.28           56.41     53.08

56.00    64.00     59.00

41.38    51.72     45.98

64.71    82.35     72.55

37.14    51.43     42.62